## Prerequisites
Please make sure to have the prerequisites installed (follow `README.md`)

### Dependencies for this demo

In [ ]:
!uv pip install \
    "flwr[simulation]==1.17.0" "flwr-datasets>=0.5.0" \
    "scikit-learn==1.6.1" "torch==2.7.0" \
    "syft_flwr==0.1.0" \
    imblearn pandas loguru

### Some paths and constants 

In [ ]:
from pathlib import Path

SYFT_FLWR_PROJECT_PATH = Path("./fl-diabetes-prediction")
SYFTBOX_DATASET_NAME = "pima-indians-diabetes-database"
RUN_SIMULATION = True

assert SYFT_FLWR_PROJECT_PATH.exists()

## Log into the data owners' datasites

In [ ]:
import syft_rds as sy
from syft_core import Client

DS = Client.load().email
print("DS email: ", DS)

DO1 = "dknguyen.work@gmail.com"
DO2 = "dkn.work@protonmail.com"

do_client_1 = sy.init_session(host=DO1)
print("Logged into: ", do_client_1.host)

do_client_2 = sy.init_session(host=DO2)
print("Logged into: ", do_client_2.host)

do_clients = [do_client_1, do_client_2]
do_emails = [DO1, DO2]

## Explore the datasets

In [ ]:
mock_paths = []
for client in do_clients:
    dataset = client.dataset.get(name=SYFTBOX_DATASET_NAME)
    mock_paths.append(dataset.get_mock_path())
    print(f"Client {client.host}'s dataset: \n{dataset}\n")

## Bootstrapping and run `syft_flwr` simulation

In [ ]:
import syft_flwr

syft_flwr.bootstrap(SYFT_FLWR_PROJECT_PATH, aggregator=DS, datasites=do_emails)

## Run `flwr` and `syft_flwr` simulations (optional)

In [ ]:
if RUN_SIMULATION:
    !flwr run {SYFT_FLWR_PROJECT_PATH}

In [ ]:
# clean up
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"}
!rm -rf weights/

In [ ]:
if RUN_SIMULATION:
    print(f"running syft_flwr simulation with mock paths: {mock_paths}")
    syft_flwr.run(SYFT_FLWR_PROJECT_PATH, mock_paths)

## Submit jobs

In [ ]:
# clean up before submitting jobs
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fl_diabetes_prediction" / "__pycache__"}
!rm -rf {SYFT_FLWR_PROJECT_PATH / "simulation_logs"}
!rm -rf weights/

In [ ]:
for client in do_clients:
    job = client.jobs.submit(
        name="Syft Flower Experiment",
        description="Syft Flower Federated Learning Experiment",
        user_code_path=SYFT_FLWR_PROJECT_PATH,
        dataset_name=SYFTBOX_DATASET_NAME,
        tags=["federated learning", "fl", "syft_flwr", "flwr"],
        entrypoint="main.py",
    )
    print(job)

## DS starts the FL server code

In [ ]:
import os

os.environ["LOGURU_LEVEL"] = "DEBUG"
!uv run {str(SYFT_FLWR_PROJECT_PATH / "main.py")} --active